<a href="https://colab.research.google.com/github/psy794/stock_pjtt/blob/master/XGBoost(0_9100)_0726_%EC%83%88%EB%A1%9C%EB%B0%9B%EC%9D%80%ED%94%BC%ED%81%B4%EB%A1%9C_PCA__ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#구글드라이브를 mount해요

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt
import pickle

#데이터 프레임을 불러옵니다

In [3]:
# data = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/20230723dataframe_일부drop.pkl', 'rb'))
data = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/slow_k, slow_d,OBV추가피클/new_base_9.pkl', 'rb'))
data

,날짜,종목코드,corp_code,reprt_code,bsns_year,시가,고가,저가,종가,close_ema_6,...,돈육_종가,돈육_변동,육우_종가,육우_변동,대두_종가,대두_변동,대두유_종가,대두유_변동,옥수수_종가,옥수수_변동
0,2021-06-01,060310,00378363,11012,2021,2890,2970,2885,2920,2896.423812,...,118.62,1.17,149.15,-1.45,1548.50,0.32,62.39,-5.97,688.75,3.28
1,2021-06-02,060310,00378363,11012,2021,2915,2975,2830,2900,2897.445580,...,118.25,-0.31,152.32,2.13,1562.50,0.90,64.96,4.12,675.00,-2.00
2,2021-06-03,060310,00378363,11012,2021,2900,2925,2875,2900,2898.175414,...,119.38,0.96,152.95,0.41,1549.25,-0.85,63.98,-1.51,662.00,-1.93
3,2021-06-04,060310,00378363,11012,2021,2930,3120,2920,2950,2912.982439,...,119.50,0.10,149.93,-1.98,1583.75,2.23,66.51,3.95,682.75,3.13
4,2021-06-07,060310,00378363,11012,2021,3000,3150,2955,3150,2980.701742,...,119.90,0.33,150.20,0.18,1560.25,-1.48,67.24,1.10,679.25,-0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,238490,00556712,11012,2023,6710,6710,6420,6430,6502.864056,...,80.67,-1.42,207.80,0.59,1322.50,-1.40,47.76,-2.07,577.50,1.14
987996,2023-05-24,238490,00556712,11012,2023,6400,6490,6210,6320,6450.617183,...,79.80,-1.08,208.98,0.57,1324.50,0.15,48.00,0.50,587.25,1.69
987997,2023-05-25,238490,00556712,11012,2023,6340,6400,6270,6330,6416.155131,...,77.65,-2.69,208.98,0.00,1324.00,-0.04,48.52,1.08,590.75,0.60
987998,2023-05-26,238490,00556712,11012,2023,6330,6410,6300,6330,6391.539379,...,76.08,-2.02,233.93,11.94,1337.25,1.00,48.82,0.62,604.00,2.24


In [4]:
with open('/content/drive/MyDrive/Colab Notebooks/separated_datasets_sectors.pkl', 'rb') as file:
    separated_datasets_sectors = pickle.load(file)

In [5]:
data.columns

Index(['날짜', '종목코드', 'corp_code', 'reprt_code', 'bsns_year', '시가', '고가', '저가',
       '종가', 'close_ema_6', 'close_ema_12', 'close_ema_26', '거래량', '거래대금',
       '시가총액', 'btc_krw_close', 'usdtocny', 'eurtousd', 'usdtoaud', 'usdtojpy',
       'usdtokrw', '금_종가', '금_변동', '은_종가', '은_변동', '구리_종가', '구리_변동', '알루미늄_종가',
       '알루미늄_변동', '납_종가', '납_변동', '니켈_종가', '니켈_변동', '백금_종가', '백금_변동', '아연_종가',
       '아연_변동', '주석_종가', '주석_변동', '팔라듐_종가', '팔라듐_변동', 'WTI유_종가', 'WTI유_변동',
       '가솔린RBOB_종가', '가솔린RBOB_변동', '난방유_종가', '난방유_변동', '런던가스오일_종가',
       '런던가스오일_변동', '브렌트유_종가', '브렌트유_변동', '천연가스_종가', '천연가스_변동', '탄소배출권_종가',
       '탄소배출권_변동', '에탄올_종가', '에탄올_변동', '커피_종가', '커피_변동', '코코아_종가', '코코아_변동',
       '설탕_종가', '설탕_변동', '오렌지주스_종가', '오렌지주스_변동', '소맥_종가', '소맥_변동', '원면_종가',
       '원면_변동', '원목_종가', '원목_변동', '돈육_종가', '돈육_변동', '육우_종가', '육우_변동', '대두_종가',
       '대두_변동', '대두유_종가', '대두유_변동', '옥수수_종가', '옥수수_변동'],
      dtype='object')

In [6]:
separated_datasets_sectors

{'IT부품':                날짜     시가     고가     저가     종가      거래량         거래대금   등락률  \
 55328  2021-06-01  20400  20800  20100  20550   211150   4320781100  1.23   
 55329  2021-06-02  20950  21150  20700  20800   344482   7202011200  1.22   
 55330  2021-06-03  21000  21700  21000  21650   434569   9314752200  4.09   
 55331  2021-06-04  21650  22100  21350  21750   333074   7254783050  0.46   
 55332  2021-06-07  21850  21900  21050  21250   241745   5136611800 -2.30   
 ...           ...    ...    ...    ...    ...      ...          ...   ...   
 961813 2023-05-23   5550   5730   5530   5650   702527   3970889110  1.80   
 961814 2023-05-24   5650   5650   5470   5540   341692   1891318820 -1.95   
 961815 2023-05-25   5510   5650   5470   5630   432993   2448873000  1.62   
 961816 2023-05-26   5790   5950   5550   5890  1464906   8453395300  4.62   
 961817 2023-05-30   5750   6250   5690   5950  3414452  20580026590  1.02   
 
           종목코드  Close_mv5  ...    nasdaq  semicon_ind

In [7]:
sector_dfs = []

for sector_name, sector_df in separated_datasets_sectors.items():
  sector_df['업종명'] = sector_name
  sector_dfs.append(sector_df)

merged_data = pd.concat(sector_dfs)


merged_data

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드,Close_mv5,...,nasdaq,semicon_index,euro50,us10y_tsy,fast_k,fast_d,시가총액,slow_k,slow_d,업종명
55328,2021-06-01,20400,20800,20100,20550,211150,4320781100,1.23,060720,NaN,...,13736.48,3174.32,4071.75,1.615,64.285714,64.285714,486498706650,64.285714,64.285714,IT부품
55329,2021-06-02,20950,21150,20700,20800,344482,7202011200,1.22,060720,NaN,...,13756.33,3196.98,4088.50,1.591,66.666667,65.714286,492417182400,65.714286,65.142857,IT부품
55330,2021-06-03,21000,21700,21000,21650,434569,9314752200,4.09,060720,NaN,...,13614.51,3139.19,4079.24,1.625,96.875000,80.474624,512539999950,80.474624,72.405273,IT부품
55331,2021-06-04,21650,22100,21350,21750,333074,7254783050,0.46,060720,NaN,...,13814.49,3214.14,4089.38,1.560,82.500000,81.315934,514907390250,81.315934,76.106625,IT부품
55332,2021-06-07,21850,21900,21050,21250,241745,5136611800,-2.30,060720,21200.0,...,13881.72,3191.66,4097.65,1.569,57.500000,72.173324,503070438750,72.173324,74.596685,IT부품
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981079,2023-05-23,4780,4805,4740,4770,25435,121248710,-0.21,079980,4723.0,...,12560.25,3177.44,4342.38,3.698,79.032258,67.391066,164565000000,67.391066,51.804965,화학
981080,2023-05-24,4770,4770,4695,4770,21634,102170545,0.00,079980,4753.0,...,12484.16,3124.09,4263.74,3.719,79.032258,71.271463,164565000000,71.271463,58.293798,화학
981081,2023-05-25,4730,4765,4680,4750,30459,143268485,-0.42,079980,4771.0,...,12698.09,3336.72,4269.64,3.814,72.580645,71.707857,163875000000,71.707857,62.765151,화학
981082,2023-05-26,4750,4750,4640,4695,28337,132163680,-1.16,079980,4753.0,...,12975.69,3545.67,4337.50,3.810,54.838710,66.084808,161977500000,66.084808,63.871703,화학


In [8]:
exclude_columns = ['업종명','종목코드']
drop_columns = [col for col in merged_data.columns if col not in exclude_columns]
drop_columns

['날짜',
 '시가',
 '고가',
 '저가',
 '종가',
 '거래량',
 '거래대금',
 '등락률',
 'Close_mv5',
 'Close_mv10',
 'Close_mv20',
 'OBV',
 'OBV_EMA',
 'sse_composite_index',
 'csi300_index',
 'usdtocny',
 'eurtousd',
 'msci_emerging',
 'usdtoaud',
 'usdtojpy',
 'usdtokrw',
 'dow',
 'snp500',
 'nasdaq',
 'semicon_index',
 'euro50',
 'us10y_tsy',
 'fast_k',
 'fast_d',
 '시가총액',
 'slow_k',
 'slow_d']

In [9]:
merged_data = merged_data.drop(columns=drop_columns)
merged_data

,종목코드,업종명
55328,060720,IT부품
55329,060720,IT부품
55330,060720,IT부품
55331,060720,IT부품
55332,060720,IT부품
...,...,...
981079,079980,화학
981080,079980,화학
981081,079980,화학
981082,079980,화학


In [10]:
merged_data.drop_duplicates(subset=['종목코드','업종명'], keep= 'first', inplace = True)
merged_data

,종목코드,업종명
55328,060720,IT부품
81510,091340,IT부품
83980,151910,IT부품
152152,190510,IT부품
155610,267320,IT부품
...,...,...
973674,298050,화학
974168,298020,화학
974662,298000,화학
975156,093370,화학


In [ ]:
# 결과를 데이터프레임을 CSV 파일로 저장
merged_data.to_csv('/content/drive/MyDrive/Colab Notebooks/종목코드별업종.csv', index=False)

In [11]:
# data에 업종 병합
data = data.merge(merged_data[['업종명','종목코드']], on='종목코드', how='left')
data

,날짜,종목코드,corp_code,reprt_code,bsns_year,시가,고가,저가,종가,close_ema_6,...,돈육_변동,육우_종가,육우_변동,대두_종가,대두_변동,대두유_종가,대두유_변동,옥수수_종가,옥수수_변동,업종명
0,2021-06-01,060310,00378363,11012,2021,2890,2970,2885,2920,2896.423812,...,1.17,149.15,-1.45,1548.50,0.32,62.39,-5.97,688.75,3.28,기계·장비
1,2021-06-02,060310,00378363,11012,2021,2915,2975,2830,2900,2897.445580,...,-0.31,152.32,2.13,1562.50,0.90,64.96,4.12,675.00,-2.00,기계·장비
2,2021-06-03,060310,00378363,11012,2021,2900,2925,2875,2900,2898.175414,...,0.96,152.95,0.41,1549.25,-0.85,63.98,-1.51,662.00,-1.93,기계·장비
3,2021-06-04,060310,00378363,11012,2021,2930,3120,2920,2950,2912.982439,...,0.10,149.93,-1.98,1583.75,2.23,66.51,3.95,682.75,3.13,기계·장비
4,2021-06-07,060310,00378363,11012,2021,3000,3150,2955,3150,2980.701742,...,0.33,150.20,0.18,1560.25,-1.48,67.24,1.10,679.25,-0.51,기계·장비
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,238490,00556712,11012,2023,6710,6710,6420,6430,6502.864056,...,-1.42,207.80,0.59,1322.50,-1.40,47.76,-2.07,577.50,1.14,반도체
987996,2023-05-24,238490,00556712,11012,2023,6400,6490,6210,6320,6450.617183,...,-1.08,208.98,0.57,1324.50,0.15,48.00,0.50,587.25,1.69,반도체
987997,2023-05-25,238490,00556712,11012,2023,6340,6400,6270,6330,6416.155131,...,-2.69,208.98,0.00,1324.00,-0.04,48.52,1.08,590.75,0.60,반도체
987998,2023-05-26,238490,00556712,11012,2023,6330,6410,6300,6330,6391.539379,...,-2.02,233.93,11.94,1337.25,1.00,48.82,0.62,604.00,2.24,반도체


In [14]:
data_dtype = data.dtypes.to_string()
print(data_dtype)

날짜                object
종목코드              object
corp_code         object
reprt_code        object
bsns_year         object
시가                 int32
고가                 int32
저가                 int32
종가                 int32
close_ema_6      float64
close_ema_12     float64
close_ema_26     float64
거래량                int32
거래대금               int64
시가총액               int64
btc_krw_close    float64
usdtocny         float64
eurtousd         float64
usdtoaud         float64
usdtojpy         float64
usdtokrw         float64
금_종가             float64
금_변동             float64
은_종가             float64
은_변동             float64
구리_종가            float64
구리_변동            float64
알루미늄_종가          float64
알루미늄_변동          float64
납_종가             float64
납_변동             float64
니켈_종가            float64
니켈_변동            float64
백금_종가            float64
백금_변동            float64
아연_종가            float64
아연_변동            float64
주석_종가            float64
주석_변동            float64
팔라듐_종가           float64


In [13]:
data.columns

Index(['날짜', '종목코드', 'corp_code', 'reprt_code', 'bsns_year', '시가', '고가', '저가',
       '종가', 'close_ema_6', 'close_ema_12', 'close_ema_26', '거래량', '거래대금',
       '시가총액', 'btc_krw_close', 'usdtocny', 'eurtousd', 'usdtoaud', 'usdtojpy',
       'usdtokrw', '금_종가', '금_변동', '은_종가', '은_변동', '구리_종가', '구리_변동', '알루미늄_종가',
       '알루미늄_변동', '납_종가', '납_변동', '니켈_종가', '니켈_변동', '백금_종가', '백금_변동', '아연_종가',
       '아연_변동', '주석_종가', '주석_변동', '팔라듐_종가', '팔라듐_변동', 'WTI유_종가', 'WTI유_변동',
       '가솔린RBOB_종가', '가솔린RBOB_변동', '난방유_종가', '난방유_변동', '런던가스오일_종가',
       '런던가스오일_변동', '브렌트유_종가', '브렌트유_변동', '천연가스_종가', '천연가스_변동', '탄소배출권_종가',
       '탄소배출권_변동', '에탄올_종가', '에탄올_변동', '커피_종가', '커피_변동', '코코아_종가', '코코아_변동',
       '설탕_종가', '설탕_변동', '오렌지주스_종가', '오렌지주스_변동', '소맥_종가', '소맥_변동', '원면_종가',
       '원면_변동', '원목_종가', '원목_변동', '돈육_종가', '돈육_변동', '육우_종가', '육우_변동', '대두_종가',
       '대두_변동', '대두유_종가', '대두유_변동', '옥수수_종가', '옥수수_변동', '업종명'],
      dtype='object')

In [15]:
# 업종명을 기준으로 데이터프레임을 그룹화합니다.
grouped_data_sectors = data.groupby('업종명')

# 그룹화된 데이터프레임을 업종명별로 따로 저장합니다.
separated_datasets_sectors = {sector: group for sector, group in grouped_data_sectors}

In [16]:
# 업종명을 기준으로 데이터프레임을 그룹화합니다.
grouped_data_sectors = data.groupby('업종명')

# 그룹화된 데이터프레임을 업종명별로 따로 저장합니다.
separated_datasets_sectors = {sector: group for sector, group in grouped_data_sectors}

In [17]:
dict_data = separated_datasets_sectors

In [175]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
import pandas as pd

industries = list(dict_data.keys())  # 전체 업종 선택
window_size = 3  # 윈도우 사이즈를 1로 설정

result_df = pd.DataFrame()  # 결과를 저장할 데이터프레임 초기화
n_components = 8  # 여기에 원하는 주성분의 수를 지정

# 각 업종에 대해 모델을 학습하고 테스트
for industry in industries:
    print(f"Processing industry: {industry}")
    codes = dict_data[industry]['종목코드'].unique()

    # 각 종목코드에 대해 모델을 학습하고 테스트
    for code in codes:
        print(f"Processing code: {code}")
        df = dict_data[industry][dict_data[industry]['종목코드'] == code].copy()

        df['날짜'] = pd.to_datetime(df['날짜'])

        df.set_index('날짜', inplace=True)

        # 종가를 제외한 모든 피처 선택 (불필요한 열 제거)
        selected_vars = df.drop(['업종명', '종목코드'], axis=1)

        # 윈도우 사이즈와 롤링 윈도우 적용
        moving_vars = selected_vars.rolling(window=window_size).mean().shift(1)

        # 예측 대상인 종가를 추가
        moving_vars['종가'] = df['종가'].shift(-1)

        moving_vars = moving_vars.dropna()

        train = moving_vars[(moving_vars.index >= '2021-06-01') & (moving_vars.index <= '2023-01-01')]
        test = moving_vars[(moving_vars.index >= '2023-05-25') & (moving_vars.index <= '2023-05-29')]

        # 테스트 데이터셋이 비어있지 않은 경우에만 모델 학습 및 예측을 수행
        if not test.empty:
            X_train = train.drop('종가', axis=1)
            y_train = train['종가']
            X_test = test.drop('종가', axis=1)
            y_test = test['종가']

            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            pca = PCA(n_components=n_components)
            X_train_pca = pca.fit_transform(X_train_scaled)
            X_test_pca = pca.transform(X_test_scaled)

            model = xgb.XGBRegressor(objective='reg:squarederror')

            print(f'Training model for code {code}...')
            model.fit(X_train_pca, y_train)

            print(f'Predicting for code {code}...')
            predictions = model.predict(X_test_pca)

            # 5월30일의 예측 종가와 5일간의 평균 종가 차이 계산
            average_close = y_test.mean()
            difference = average_close - predictions[-1]

            print(f"Code: {code}, Predicted Close on May 30: {predictions[-1]}, Average Close from May 20 to May 29: {average_close}, Difference: {difference}")

            ## 결과를 데이터프레임에 추가 (예측 결과가 있을 때만 추가)
            if not np.isnan(predictions[-1]):
                result_row = pd.DataFrame({'종목코드': [code], 'Difference': [difference], '예측종가_5월30일': [predictions[-1]]})
                if result_df.empty:
                    result_df = result_row
                else:
                    result_df = pd.concat([result_df, result_row])


# 결과를 데이터프레임을 CSV 파일로 저장
# result_df.to_csv('result.csv', index=False)
result_df


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Predicting for code 002150...
Code: 002150, Predicted Close on May 30: 8263.859375, Average Close from May 20 to May 29: 8630.0, Difference: 366.140625
Processing code: 102260
Training model for code 102260...
Predicting for code 102260...
Code: 102260, Predicted Close on May 30: 4824.70654296875, Average Close from May 20 to May 29: 4610.0, Difference: -214.70654296875
Processing code: 032350
Training model for code 032350...
Predicting for code 032350...
Code: 032350, Predicted Close on May 30: 12202.9248046875, Average Close from May 20 to May 29: 11175.0, Difference: -1027.9248046875
Processing code: 286940
Training model for code 286940...
Predicting for code 286940...
Code: 286940, Predicted Close on May 30: 23184.296875, Average Close from May 20 to May 29: 27925.0, Difference: 4740.703125
Processing code: 007120
Training model for code 007120...
Predicting for code 007120...
Code: 007120, Predicted Close on May 30: 2247.737060546875, Average 

,종목코드,Difference,예측종가_5월30일
0,060720,-655.111328,21230.111328
0,091340,330.058105,3789.941895
0,151910,-662.405640,1170.905640
0,190510,2995.531250,13359.468750
0,267320,450.813477,3031.686523
...,...,...,...
0,298050,17382.906250,428117.093750
0,298020,35919.812500,347580.187500
0,298000,-1228.250000,101578.250000
0,093370,-1847.374023,15367.374023


In [176]:
# 주가 대비 변동률 계산
result_df['주가대비변동률'] = result_df['Difference'] / result_df['예측종가_5월30일']


# 순위 매기기
result_df['순위'] = result_df['주가대비변동률'].rank(ascending=False)

In [177]:
result_df

,종목코드,Difference,예측종가_5월30일,주가대비변동률,순위
0,060720,-655.111328,21230.111328,-0.030858,1309.0
0,091340,330.058105,3789.941895,0.087088,675.0
0,151910,-662.405640,1170.905640,-0.565721,1993.0
0,190510,2995.531250,13359.468750,0.224225,302.0
0,267320,450.813477,3031.686523,0.148701,460.0
...,...,...,...,...,...
0,298050,17382.906250,428117.093750,0.040603,900.0
0,298020,35919.812500,347580.187500,0.103343,599.0
0,298000,-1228.250000,101578.250000,-0.012092,1224.0
0,093370,-1847.374023,15367.374023,-0.120214,1702.0


In [178]:
#주가대비변동률이 음수이고 5일간 평균 종가와 예측 종가의 차이가 큰 마이너스 값을 하위200개에 배치하고자 함
# 주가대비변동률이 음수이고 5일간 평균 종가와 예측 종가의 차이가 큰 마이너스 값인 경우를 선택
filtered_df = result_df[(result_df['주가대비변동률'] < 0) & (result_df['Difference'] < 0)]

# 순위를 매기기 위해 'Difference' 컬럼을 기준으로 내림차순 정렬
filtered_df = filtered_df.sort_values('Difference', ascending=False)

# 순위 컬럼 추가 및 순위 매김
filtered_df['순위'] = filtered_df['Difference'].rank(ascending=False)

# 결과 확인
print(filtered_df[['종목코드', 'Difference', '순위']])

      종목코드     Difference     순위
0   006200      -0.849243    1.0
0   006570      -0.969482    2.0
0   001620      -2.001892    3.0
0   033200      -2.229492    4.0
0   085310      -2.774597    5.0
..     ...            ...    ...
0   278280  -78010.500000  846.0
0   000670  -84143.625000  847.0
0   017390  -91475.640625  848.0
0   036570 -112974.937500  849.0
0   051900 -129932.562500  850.0

[850 rows x 3 columns]


In [179]:
# down 200에 하위 200개 저장
down_200 = filtered_df.tail(200)

# down 200의 순위를 다시 매김
down_200['Rank'] = range(1801, 2001)

# 최종 순위를 오름차순으로 정렬
final_ranking = pd.concat([filtered_df.head(len(filtered_df) - 200), down_200])

# 최종 순위를 하위 200개만 저장
final_ranking = final_ranking.tail(200)

# 결과 확인
final_ranking

<ipython-input-179-7460fe3fb49e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  down_200['Rank'] = range(1801, 2001)


,종목코드,Difference,예측종가_5월30일,주가대비변동률,순위,Rank
0,244460,-2194.402832,5574.402832,-0.393657,651.0,1801.0
0,007330,-2201.430664,11711.430664,-0.187973,652.0,1802.0
0,264450,-2202.340820,15372.340820,-0.143266,653.0,1803.0
0,214320,-2212.421875,41462.421875,-0.053360,654.0,1804.0
0,002170,-2227.218750,52977.218750,-0.042041,655.0,1805.0
...,...,...,...,...,...,...
0,278280,-78010.500000,264860.500000,-0.294534,846.0,1996.0
0,000670,-84143.625000,618643.625000,-0.136013,847.0,1997.0
0,017390,-91475.640625,174125.640625,-0.525343,848.0,1998.0
0,036570,-112974.937500,441474.937500,-0.255903,849.0,1999.0


In [180]:
final_ranking['Rank'] = final_ranking['Rank'].astype(int)
final_ranking

,종목코드,Difference,예측종가_5월30일,주가대비변동률,순위,Rank
0,244460,-2194.402832,5574.402832,-0.393657,651.0,1801
0,007330,-2201.430664,11711.430664,-0.187973,652.0,1802
0,264450,-2202.340820,15372.340820,-0.143266,653.0,1803
0,214320,-2212.421875,41462.421875,-0.053360,654.0,1804
0,002170,-2227.218750,52977.218750,-0.042041,655.0,1805
...,...,...,...,...,...,...
0,278280,-78010.500000,264860.500000,-0.294534,846.0,1996
0,000670,-84143.625000,618643.625000,-0.136013,847.0,1997
0,017390,-91475.640625,174125.640625,-0.525343,848.0,1998
0,036570,-112974.937500,441474.937500,-0.255903,849.0,1999


In [181]:

# Difference, 예측종가_5월30일, Rank, 주가대비변동률, 순위 컬럼 제거
final_ranking = final_ranking.drop(['Difference', '예측종가_5월30일','주가대비변동률', '순위'], axis=1)

final_ranking['종목코드'] = final_ranking['종목코드'].astype(str).str.zfill(6)
final_ranking['종목코드'] = np.where(final_ranking['종목코드'].str.len() == 6, 'A' + final_ranking['종목코드'], final_ranking['종목코드'])
final_ranking

,종목코드,Rank
0,A244460,1801
0,A007330,1802
0,A264450,1803
0,A214320,1804
0,A002170,1805
...,...,...
0,A278280,1996
0,A000670,1997
0,A017390,1998
0,A036570,1999


In [182]:
final_ranking.rename(columns={'Rank': '순위'}, inplace=True)


In [183]:
result_df

,종목코드,Difference,예측종가_5월30일,주가대비변동률,순위
0,060720,-655.111328,21230.111328,-0.030858,1309.0
0,091340,330.058105,3789.941895,0.087088,675.0
0,151910,-662.405640,1170.905640,-0.565721,1993.0
0,190510,2995.531250,13359.468750,0.224225,302.0
0,267320,450.813477,3031.686523,0.148701,460.0
...,...,...,...,...,...
0,298050,17382.906250,428117.093750,0.040603,900.0
0,298020,35919.812500,347580.187500,0.103343,599.0
0,298000,-1228.250000,101578.250000,-0.012092,1224.0
0,093370,-1847.374023,15367.374023,-0.120214,1702.0


In [184]:
result_df = result_df.reset_index(drop=True)


In [185]:
# 주가대비변동률에 대한 조건 포함해서 5일간 평균 종가와 예측 종가 차이가 양수인것들 기준으로 순위매겨서 상위200개만 추출
# threshold를 적게 할수록 주가의 안정성이 높은 종목들이 상위권에 포함될 가능성이 높아짐

threshold = 0.1
result_df['rank'] = result_df[(result_df['주가대비변동률'] < threshold) & (result_df['Difference'] > 0)].rank(ascending=False)['Difference']


In [186]:

# NaN 값을 가진 rank를 하위로 배치하고, rank를 오름차순으로 정렬
result_df['rank'] = result_df['rank'].fillna(result_df['rank'].max() + 1)
result_df['rank'] = result_df['rank'].astype(int)
result_df = result_df.sort_values(by='rank')
# 상위 200개 행만 유지하고 나머지 행은 제거
result_df = result_df.head(200)
result_df

,종목코드,Difference,예측종가_5월30일,주가대비변동률,순위,rank
1847,051910,55427.312500,648572.687500,0.085460,682.0,1
1493,006400,29583.687500,675916.312500,0.043768,885.0,2
1474,011070,22514.937500,276235.062500,0.081506,697.0,3
693,000810,19170.703125,209579.296875,0.091472,653.0,4
814,001460,18731.593750,402018.406250,0.046594,870.0,5
...,...,...,...,...,...,...
901,053300,379.196289,4860.803711,0.078011,714.0,196
1234,023900,378.006836,12886.993164,0.029332,962.0,197
1588,260660,373.969727,5926.030273,0.063106,782.0,198
557,069080,371.435547,14693.564453,0.025279,992.0,199


In [187]:
# Difference, 예측종가_5월30일, Rank, 주가대비변동률, 순위 컬럼 제거
result_df = result_df.drop(['Difference', '예측종가_5월30일', '주가대비변동률', '순위'], axis=1)
result_df['종목코드'] = result_df['종목코드'].astype(str).str.zfill(6)
result_df['종목코드'] = np.where(result_df['종목코드'].str.len() == 6, 'A' + result_df['종목코드'], result_df['종목코드'])
result_df.rename(columns={'rank': '순위'}, inplace=True)
# 결과 확인
result_df

,종목코드,순위
1847,A051910,1
1493,A006400,2
1474,A011070,3
693,A000810,4
814,A001460,5
...,...,...
901,A053300,196
1234,A023900,197
1588,A260660,198
557,A069080,199


In [188]:
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv')
sample_submission

,종목코드,순위
0,A000020,1
1,A000040,2
2,A000050,3
3,A000070,4
4,A000080,5
...,...,...
1995,A375500,1996
1996,A378850,1997
1997,A383220,1998
1998,A383310,1999


In [189]:
# final_ranking에서 종목코드와 순위 컬럼 선택
final_ranking_subset = final_ranking[['종목코드', '순위']]

# sample_submission에 final_ranking 순위 병합
sample_submission = pd.merge(sample_submission, final_ranking_subset, on='종목코드', how='left')

# 병합된 순위 컬럼명을 수정
sample_submission.rename(columns={'순위': '순위_final_ranking'}, inplace=True)

# result_df에서 종목코드와 순위 컬럼 선택
result_df_subset = result_df[['종목코드', '순위']]

# sample_submission에 result_df 순위 병합
sample_submission = pd.merge(sample_submission, result_df_subset, on='종목코드', how='left')

# 병합된 순위 컬럼명을 수정
sample_submission.rename(columns={'순위': '순위_result_df'}, inplace=True)

# '순위_y'와 '순위_result_df' 컬럼의 NaN을 0으로 채우기
sample_submission['순위_y'].fillna(0, inplace=True)
sample_submission['순위_result_df'].fillna(0, inplace=True)

# '순위_y'와 '순위_result_df' 컬럼을 더한 값을 '순위' 컬럼에 추가
sample_submission['순위'] = sample_submission['순위_y'] + sample_submission['순위_result_df']

# '순위_x', '순위_y', '순위_result_df' 컬럼 삭제
sample_submission.drop(['순위_x', '순위_y', '순위_result_df'], axis=1, inplace=True)

# '순위' 컬럼을 int형으로 변환
sample_submission['순위'] = sample_submission['순위'].astype(int)

# '순위' 컬럼에 0이 포함된 행들에 201부터 1800까지의 숫자를 넣어줌
mask = sample_submission['순위'] == 0
sample_submission.loc[mask, '순위'] = range(201, 1801)


# sample_submission.to_csv('/content/drive/MyDrive/Colab Notebooks/0724_winsize1다시_featureimportance001_columnsdrop_pluscolumns_durationplus.csv', index=False) #0.8769파일명
sample_submission.to_csv('/content/drive/MyDrive/Colab Notebooks/0726_ncompo8.csv', index=False)

# 결과 확인
sample_submission

,종목코드,순위
0,A000020,201
1,A000040,202
2,A000050,203
3,A000070,17
4,A000080,1860
...,...,...
1995,A375500,1797
1996,A378850,1798
1997,A383220,1970
1998,A383310,1799
